<a href="https://colab.research.google.com/github/zaitsevIV/hse_gp2_scrap-api/blob/develop/Selenium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Удаляем старые версии Chrome и Chromedriver
!apt-get remove -y chromium-browser chromium-chromedriver
!rm -rf /usr/bin/chromedriver /usr/bin/chromium-browser

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following packages were automatically installed and are no longer required:
  apparmor libfuse3-3 liblzo2-2 snapd squashfs-tools systemd-hwe-hwdb udev
Use 'apt autoremove' to remove them.
The following packages will be REMOVED:
  chromium-browser chromium-chromedriver
0 upgraded, 0 newly installed, 2 to remove and 48 not upgraded.
After this operation, 243 kB disk space will be freed.
(Reading database ... 125567 files and directories currently installed.)
Removing chromium-chromedriver (1:85.0.4183.83-0ubuntu2.22.04.1) ...
Removing chromium-browser (1:85.0.4183.83-0ubuntu2.22.04.1) ...
Processing triggers for mailcap (3.70+nmu1ubuntu1) ...
Processing triggers for hicolor-icon-theme (0.17-2) ...


In [ ]:
# Устанавливаем зависимости
!apt-get update -y
!apt-get install -y chromium-browser chromium-chromedriver
!pip install selenium webdriver-manager

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:6 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:11 https://dl.google.com/linux/chrome/deb stable InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading

In [ ]:
basck=pd.read_csv('basck_vacancies.csv')
basck

,city,position,salary,link
0,"Уфа,",Слесарь-ремонтник,40 000 — 44 000 руб.,https://bashkortostan.rabota.ru/vacancy/500909...


In [ ]:
# Опции браузера
options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-gpu")
options.add_argument("--disable-extensions")

# Запуск браузера
service = Service()
driver = webdriver.Chrome(service=service, options=options)
wait = WebDriverWait(driver, 10)

def parse_vacancy_page():
    time.sleep(2)
    try:
        city = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'vacancy-requirements__city'))).text.strip()
    except Exception:
        city = None

    try:
        position = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'vacancy-card__title'))).text.strip()
    except Exception:
        position = None

    try:
        salary = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'vacancy-card__salary'))).text.strip()
    except Exception:
        salary = None

    return [city, position, salary, driver.current_url]

def scrape_vacancies(base_url, max_pages=10, search_query=None):
    all_vacancies = []

    for current_page in tqdm(range(1, max_pages + 1)):
        url = f'{base_url}?page={current_page}'
        driver.get(url)

        if search_query:
            try:
                search_box = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'search-input')))
                search_box.clear()
                search_box.send_keys(search_query)
                search_box.send_keys(Keys.RETURN)
                time.sleep(3)
            except Exception:
                pass

        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)

        try:
            job_links = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'vacancy-preview-card__title_border')))
            for job in job_links:
                try:
                    job_url = job.get_attribute("href")  # Получаем ссылку
                    driver.get(job_url)  # Открываем в той же вкладке
                    time.sleep(2)
                    all_vacancies.append(parse_vacancy_page())
                    driver.get(url)  # Возвращаемся на список вакансий
                    time.sleep(2)
                except Exception:
                    continue
        except Exception:
            continue

    return all_vacancies

# Запуск парсинга
basck_data = scrape_vacancies('https://bashkortostan.rabota.ru/vacancy', max_pages=20, search_query="Программист")
chelyb_data = scrape_vacancies('https://chelyabinskaya.rabota.ru/vacancy', max_pages=20, search_query="Аналитик")

# Сохранение данных
basck_df = pd.DataFrame(basck_data, columns=['city', 'position', 'salary', 'link'])
chelyb_df = pd.DataFrame(chelyb_data, columns=['city', 'position', 'salary', 'link'])

basck_df.to_csv('basck_vacancies.csv', sep=',', encoding='utf-8', index=False)
chelyb_df.to_csv('chelyb_vacancies.csv', sep=',', encoding='utf-8', index=False)

# Закрываем браузер
driver.quit()


100%|██████████| 20/20 [11:00<00:00, 33.02s/it]
